In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#!pip install python-binance -qq
!pip install -qq wandb
#!conda install -c conda-forge sktime-all-extras -y
!pip install sktime
!pip install -qq --upgrade tsfresh
!pip install scipy==1.5

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
import math
import numpy as np
#from numba import cuda
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from matplotlib.pyplot import figure
import statistics

from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tqdm.notebook import tqdm

#Importing the Libraries
%matplotlib inline
import matplotlib. pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.dates as mandates
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import adam_v2, adamax_v2, adadelta_v2, adagrad_v2, rmsprop_v2
#from keras.optimizers import Adam, Adamax, Adadelta, Adagrad
from keras.models import load_model
from keras.layers import LSTM, GRU, Flatten,BatchNormalization
from tensorflow.keras import regularizers
from keras.utils.vis_utils import plot_model
import keras
import pickle

import tsfresh
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
#from tsfresh.feature_extraction.settings import Feature_Extraction_Settings
#from sktime.transformers.series_as_features.summarize import TSFreshFeatureExtractor

from sklearn.metrics import classification_report, recall_score, precision_score

#from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.metrics import classification_report
from datetime import datetime

from IPython.display import clear_output 
from scipy import signal
from sklearn.metrics import classification_report, recall_score, precision_score

from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
root = '/content/drive/MyDrive/'
path = root+'generate_dataset/ETHBUSD/'

In [ ]:
#path = '/home/kurniawan/dataset/'

## **Feature**

In [ ]:
df = pd.read_csv(path+"ETHBUSD-5m-data.csv", parse_dates=['timestamp'])
del df['close_time']
del df['quote_av']
del df['tb_base_av']
del df['tb_quote_av']
del df['ignore']

# volume / trades
vt_mult = 1
df['vt'] = ( df['volume']/df['trades'] ) * vt_mult
del df['trades']
del df['volume']

# precentage change
pct_mult = 2500
df['pct_change'] = df['close'].pct_change(1)
df['pct_change'] = (df['pct_change'] +1 ) * 100
#df['pct_change'] = ( df['pct_change'] + 1 ) * pct_mult
df.dropna(inplace=True)

# movement
fee = 0.0016
move_mult = 1
df['move'] = df['pct_change'].apply(lambda x: 1 if x > (fee+1)*100 else 0)
#del df['pct_change_ori']
df['move'] = df['move'] * move_mult

#
timeframe_day = int(( 60 / 5 ) * 24)
timeframe = int( timeframe_day * 365.25/13) +1
df = df.iloc[-(timeframe):-1]
df.reset_index(inplace=True)
del df['index']
df

### **tsfresh**

In [ ]:
features = ['open', 'high', 'low', 'vt', 'pct_change', 'move']
target = 'close'

In [ ]:
# 2D dataframe
#X = pd.DataFrame()
#for i, ts in tqdm(enumerate( np.array(df[features]) ), total=len(df[features])):
#    data = [[x, i] for x in ts]
#    X = X.append(data, ignore_index=True)
#X.columns = ['value', 'id']

In [ ]:
X = df[features]
X['id'] = np.arange(0,len(df))
#X = df.copy()
X

In [ ]:
from tsfresh import extract_relevant_features

features_filtered_direct = extract_relevant_features(X, df[target], column_id='id')
impute(features_filtered_direct)
features_filtered_direct.head()

In [ ]:
sf = select_features(features_filtered_direct, df[target])
sf.head()

In [ ]:
with open(path+'features.pickle', 'wb') as handle:
    pickle.dump(sf[:1], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(path+'features.pickle', 'rb') as handle:
    col = pickle.load(handle)

In [ ]:
kind_to_fc_parameters = tsfresh.feature_extraction.settings.from_columns(col)

In [ ]:
ef = extract_features(X, column_id = 'id',
                      kind_to_fc_parameters = kind_to_fc_parameters)
ef.head()

In [ ]:
sf.columns == ef[sf.columns].columns 

### **PCA**

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(ef[col])
principalDf = pd.DataFrame(principalComponents, columns=['1', '2'])
principalDf

In [ ]:
pca.explained_variance_ratio_

In [ ]:
sum(pca.explained_variance_ratio_)

### **Scaler**

In [ ]:
scaler1 = MinMaxScaler(feature_range=(1700,2600))
scaler2 = MinMaxScaler(feature_range=(1700,2600))

scaler1.fit( pd.DataFrame( principalDf['1']) )
scaler2.fit( pd.DataFrame( principalDf['2']) )

In [ ]:
data1 = scaler1.transform( pd.DataFrame( principalDf['1']) )
data2 = scaler2.transform( pd.DataFrame( principalDf['2']) )

## **Prepro** 

In [ ]:
def create_dataset(data, look_back):
  return 0

In [ ]:
features = ['open', 'high', 'low', 'vt', 'pct_change', 'move' , 'pca1', 'pca2']
target = 'close'
df['pca1'] = data1
df['pca2'] = data2
df

In [ ]:
y_train, y_test, x_train, x_test = temporal_train_test_split(y = np.array(df[target]),
                                                             X = np.array(df[features]),
                                                             #test_size = int(timeframe_day * 7))
                                                             test_size = 1440 ) 

In [ ]:
look_back = 1
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], look_back))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], look_back))

In [ ]:
x_train.shape

In [ ]:
#del df

## **Model**

In [ ]:
nodes = 80
lookback = 1
bias = True
#nodes = config.nodes

model = Sequential()

model.add(GRU(int(nodes/10*lookback), input_shape=(x_train.shape[1],x_train.shape[2]), use_bias=bias, activation = "elu", 
              return_sequences=True,  activity_regularizer='l1'))
model.add(GRU(int(nodes*lookback), use_bias=bias, activation = "elu", return_sequences=True, activity_regularizer='l1'))
model.add(GRU(int(nodes*lookback), use_bias=bias, activation = "elu", return_sequences=True, activity_regularizer='l1'))
model.add(GRU(int(nodes*lookback), use_bias=bias, activation = "elu", return_sequences=True, activity_regularizer='l1'))
model.add(GRU((nodes*lookback), use_bias=bias, activation = "elu", activity_regularizer='l1'))
#model.add(Dropout(0.001))
model.add(Flatten())
model.add(Dense(int(nodes*lookback),  use_bias=bias, activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.8),  use_bias=bias, activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.6),  use_bias=bias, activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.4),  use_bias=bias, activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.2),  activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.1),  activation = "elu", activity_regularizer='l1'))
model.add(Dense(1, use_bias=bias, activation = "elu", activity_regularizer='l1'))


In [ ]:
nodes = 80
lookback = 1
bias = True
#nodes = config.nodes

model = Sequential()

model.add(GRU(int(nodes*0.1), input_shape=(x_train.shape[1],x_train.shape[2]), use_bias=bias, activation = "elu", 
              return_sequences=True,  activity_regularizer='l1'))
model.add(GRU(int(nodes*0.2), use_bias=bias, activation = "elu", return_sequences=True, activity_regularizer='l1'))
model.add(GRU(int(nodes*0.4), use_bias=bias, activation = "elu", return_sequences=True, activity_regularizer='l1'))
model.add(GRU(int(nodes*0.6), use_bias=bias, activation = "elu", return_sequences=True, activity_regularizer='l1'))
model.add(GRU(int(nodes*0.8), use_bias=bias, activation = "elu", return_sequences=True, activity_regularizer='l1'))
model.add(GRU(int(nodes), use_bias=bias, activation = "elu", return_sequences=True, activity_regularizer='l1'))
model.add(GRU((nodes*lookback), use_bias=bias, activation = "elu", activity_regularizer='l1'))
#model.add(Dropout(0.001))
model.add(Flatten())
model.add(Dense(int(nodes*lookback),  use_bias=bias, activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback),  use_bias=bias, activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.8),  use_bias=bias, activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.6),  use_bias=bias, activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.4),  use_bias=bias, activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.2),  activation = "elu", activity_regularizer='l1'))
model.add(Dense(int(nodes*lookback*0.1),  activation = "elu", activity_regularizer='l1'))
model.add(Dense(1, use_bias=bias, activation = "elu", activity_regularizer='l1'))

### **Callbacks**

In [ ]:
initial_learning_rate = 0.001
epochs = 100
decay = initial_learning_rate / (epochs/10)
def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)

In [ ]:
filepath = path+"weight.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint,
                  LearningRateScheduler(lr_time_based_decay, verbose=1)] 

### **Fit**

In [ ]:
model.compile(loss='mean_absolute_error', optimizer= 'adam')
model.fit(x_train, y_train,
          epochs = epochs, batch_size = int(len(x_train)/2),
          #validation_data=(testX, y_test),
          #validation_data=(trainX[:5], y_train[:5]),
          verbose = 1, 
          shuffle = False,
          callbacks = callbacks_list)

## **Eval**

In [ ]:
model = load_model(filepath)

### **Predict**

In [ ]:
y_pred = model.predict(x_test)
y_pred = [x[0] for x in y_pred]

In [ ]:
len(y_pred)

### **Profitability**

In [ ]:
eval = pd.DataFrame(y_test, columns = ['actual'])
eval['pred'] = y_pred

In [ ]:
moe = 0 
for i, row in eval.iterrows():
  moe+= row['actual'] - row['pred']
moe = moe/len(eval)
eval['pred_moe'] = eval['pred'] + moe

eval['actual_pct_change'] = eval['actual'].pct_change(1)
eval['pred_pct_change'] = eval['pred'].pct_change(1)
eval['pred_moe_pct_change'] = eval['pred_moe'].pct_change(1)
eval.dropna(inplace=True)

eval['actual_move'] = eval['actual_pct_change'].apply(lambda x: 1 if x>fee else 0)
eval['pred_move'] = eval['pred_pct_change'].apply(lambda x: 1 if x>fee else 0)
eval['pred_moe_move'] = eval['pred_moe_pct_change'].apply(lambda x: 1 if x>fee else 0)

eval

In [ ]:
figure(figsize=(25, 7), dpi=80)
plt.plot(eval['actual'], label='actual')
#plt.plot(eval['pred'], label='pred')
plt.plot(eval['pred_moe'], label='pred_moe')
plt.legend(loc='best')

In [ ]:
actual_profit = eval[eval['actual_move']==1]
init_asset = 100
asset = 0
for i, row in actual_profit.iterrows():
  asset+= init_asset * row['actual_pct_change']
print((asset/init_asset)*100)

profit = eval[eval['pred_move']==1]
init_asset = 100
asset = 0
for i, row in profit.iterrows():
  asset+= init_asset * row['actual_pct_change']
profit

In [ ]:
(asset/init_asset)*100

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(eval['actual_move'], eval['pred_move']))